# Récupération des données historiques de l'indice VIX et de l'indice de performance du SP500 

**Projet Python - 2A ENSAE** . 

Elena Loumagne / Jérémie Darracq 



## Introduction 
Ce notebook a été crée pour webscrapper les données de l'indice VIX et de l'indice de performance du SP500. Pour obtenir le jeu de données entier, nous allons utiliser la librairie **selenium** qui nous permettra de naviguer sur page HTML. 


## Packages utilisés 

In [31]:
#%pip install selenium 

In [32]:
import time 
import bs4
import pandas
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait 
from selenium.webdriver.support import expected_conditions as EC
from datetime import datetime, timedelta



## WEB SCRAPPING 

On crée une fonction de webscrapping utilisable sur les données du site [Investing.com](https://fr.investing.com/indices/) : 

- Étape 1 : on accède au site web en acceptant les cookies   
- Étape 2 : on choisit la période sur laquelle on veut afficher les valeurs de l'indice du VIX     
- Étape 3 : on stocke la page Html pour pouvoir ensuite extraire les données 

In [33]:
def web_scrapper(path,Xpath):
    """
    Paramètres :
    -------------

    'path : str
    Lien url de la page web 
    
    'Xpath : str
    Lien Xpath de l'objet date à modifier 

    Return :
    ---------

    'webpage : code HTML de la page webscrapée 
    """

    driver = webdriver.Chrome(ChromeDriverManager().install())
    driver.get(path)
    WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.ID, "onetrust-accept-btn-handler"))).click()

    dropdown =driver.find_element("id",'history-timeframe-selector')
    dropdown.click() 

    # on cherche les différentes options présentes dans le dropdown et on clique sur celle qu'on veut
    driver.find_element("id","react-select-2-option-0").click() # 0 -> journalier, 1 -> Hebdomadaire, 2 -> pour Mensuel 

    calendar=driver.find_element(By.CLASS_NAME,"historical-data_history-date-picker-wrapper__dDOuq")
    calendar.click()

    date_input = driver.find_element(By.XPATH,Xpath)
    date_input.clear()
    date_input.send_keys("3004-08-2005") #date à laquelle on souhaite commencer (marche pas mais permet de donner tout l'historique pour l'instant)


    validate=driver.find_element(By.CLASS_NAME,"HistoryDatePicker_arrow-icon__NwxN4")
    validate.click()
    time.sleep(6)

    time.sleep(6)

    webpage=driver.page_source # on stock le code html de la page où toutes le données sont chargées
    driver.quit()  
    print("Page Webscrappée")

    return webpage

### Webscrapping de l'indice VIX 

In [34]:
path_VIX= 'https://fr.investing.com/indices/volatility-s-p-500-historical-data'
Xpath_VIX = '//*[@id="__next"]/div/div/div/div[2]/main/div/div[5]/div/div/div[2]/div[2]/div[2]/div[2]/div[1]/div/div[1]/input'

web_page_VIX = web_scrapper(path_VIX,Xpath_VIX)


C:\Users\elelo\AppData\Local\Temp/ipykernel_14272/3617288181.py:18: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())


Page Webscrappée


### Webscrapping de l'indice de performance du SP500

In [35]:
path_SP500 = 'https://fr.investing.com/indices/us-spx-500-historical-data'
Xpath_SP500 = '//*[@id="__next"]/div/div/div/div[2]/main/div/div[5]/div/div/div[2]/div[2]/div[2]/div[2]/div[1]/div/div[1]/input'

web_page_SP500 = web_scrapper(path_SP500,Xpath_SP500)


C:\Users\elelo\AppData\Local\Temp/ipykernel_14272/3617288181.py:18: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())


Page Webscrappée


## Passage de l'HTML au Dataframe 


Maintenant que nous avons le code HTML de toutes les données, nous allons décortiquer la page grâce à la librairie BeautifulSoup.

On crée une fonction pour passer du code source Html à un Dataframe. Cette fonction récupère les valeurs de l'indice affichées dans le tableau des données historiques.

In [36]:
def Html_to_df(web_page):
    """
    'web_page : html code 
     code html de la page webscrappée
    """
    # utiliser le package BeautifulSoup qui "comprend" les balises 
    page = bs4.BeautifulSoup(web_page, "lxml")

    # on identifie le tableau des indices vix et SP500 : c'est le premier qui a cette classe "datatable_table__D_jso datatable_table--border__B_zW0 datatable_table--mobile-basic__W2ilt datatable_table--freeze-column__7YoIE"
    tableau_indice = page.find('table', {'class' : 'datatable_table__D_jso datatable_table--border__B_zW0 datatable_table--mobile-basic__W2ilt datatable_table--freeze-column__7YoIE'})
    table_body = tableau_indice.find('tbody')

    # on recherche toutes les lignes du tableau avec la balise "tr"
    rows = table_body.find_all('tr')

    # on obtient une liste où chaque élément est une des lignes du tableau
    cols = rows[0].find_all('td')
    dico_indice = dict()
    for row in rows:
        cols = row.find_all('td')
        cols = [ele.text.strip() for ele in cols]
        if len(cols) > 0 : 
            dico_indice[cols[0]] = cols[1:]
    
    # On transforme le dictionnaire en DataFrame  
    data_indice = pandas.DataFrame.from_dict(dico_indice,orient='index')        
    return data_indice 


    

### DataFrame de l'indice du VIX

In [94]:
data_vix = Html_to_df(web_page_VIX)

### DataFrame de l'indice de performance du SP500

In [95]:
data_SP500 = Html_to_df(web_page_SP500)

## Modification des DataFrames

In [96]:
def replace_comma_to_float(text):
    return float(text.replace(',','.'))
    
def replace_point(text):
    return text.replace('.','')


# Fonction principale qui nettoie le dataframe 

def clear_df(data,indice):
    ## On renomme les colonnes du dataframe
    data = data.rename(columns={0:"dernier_"+indice,1:'ouverture',2:'higher',3:'lower',4:'volatilité',5:'variation'})
    data = data.drop(columns=['volatilité'])
    
    ## Pour pouvoir utiliser nos données, il faut transformer les indices en *float*
    if indice=='SP500':
        data["dernier_"+indice]=data["dernier_"+indice].apply(replace_point)
        data["ouverture"]=data["ouverture"].apply(replace_point)
        data["higher"]=data["higher"].apply(replace_point)
        data["lower"]=data["lower"].apply(replace_point)

    data["dernier_"+indice]=data["dernier_"+indice].apply(replace_comma_to_float)
    data["ouverture"]=data["ouverture"].apply(replace_comma_to_float)
    data["higher"]=data["higher"].apply(replace_comma_to_float)
    data["lower"]=data["lower"].apply(replace_comma_to_float)
    
    ## On modifie le format de la date 
    data["date"]=data.index
    data.reset_index(drop=True, inplace=True)
    data["date"]=data['date'].apply(lambda x : datetime.strptime(x, '%d/%m/%Y'))


    return data



In [97]:
data_vix = clear_df(data_vix,'VIX')
data_sp500 = clear_df(data_SP500,'SP500')

In [99]:
data_sp500["date"]=data_sp500["date"].astype(str)
data_vix["date"]=data_vix["date"].astype(str)

### Ajout des valeurs pour les weekends 

In [100]:
def time_travel_day(now, day, dtformat):
        now = datetime.strptime(now, dtformat)
        back_in_time = now - timedelta(days=day)
        return back_in_time.strftime(dtformat)

def append_data(dernier,indice,ouverture,higher,lower,variation,date):

        data ={"dernier_"+indice:dernier,
        "ouverture":ouverture,
        "higher":higher,
        "lower":lower,
        "variation":variation,
        "date":date                
        }
        return data


def ajout_weekend(date_lundi,indice,df):

    """"
    Fonction qui permet d'ajouter les valeurs de l'indice pour le week end, l'ouverture de leur marché étant la clôture du vendredi en question et leur clôture de marché étant
    l'ouverture du lundi.

    """    
    
    dtformat = '%Y-%m-%d'  

    samedi=time_travel_day(date_lundi,2, dtformat)
    dimanche=time_travel_day(date_lundi,1,dtformat)
    vendredi=time_travel_day(date_lundi,3,dtformat)

    value_vendredi=df[df["date"]==vendredi]
    value_lundi=df[df["date"]==date_lundi]
    
    
    vendredi_close = float(value_vendredi["dernier_"+indice])
    monday_open = float(value_lundi["ouverture"])

    percent_change=100*(1-(vendredi_close/monday_open))
    percent_change=round(percent_change,2)
    str_percent_change=str(percent_change)+"%"

    data_samedi=append_data(monday_open,indice,vendredi_close,max(vendredi_close,monday_open),min(vendredi_close,monday_open),str_percent_change,samedi)
    data_dimanche=append_data(monday_open,indice,vendredi_close,max(vendredi_close,monday_open),min(vendredi_close,monday_open),str_percent_change,dimanche)
    
    df=df.append(data_samedi,ignore_index=True)
    df=df.append(data_dimanche,ignore_index=True)

    return df



In [101]:
# on rajoute les valeurs des week end sur les deux derniers mois
dtformat = '%Y-%m-%d' 

for i in range(0,65): 
    if time_travel_day(data_vix.loc[i,"date"],1,dtformat)!=data_vix.loc[i+1,"date"]:
        data_vix=ajout_weekend(data_vix.loc[i,"date"],"VIX",data_vix)
        

data_vix.sort_values("date",ascending=False,inplace=True)
data_vix=data_vix.reset_index(drop=True)


# on rajoute les valeurs des week end sur les deux derniers mois

for i in range(0,65): 
    if time_travel_day(data_sp500.loc[i,"date"],1,dtformat)!=data_sp500.loc[i+1,"date"]:
        data_sp500=ajout_weekend(data_sp500.loc[i,"date"],"SP500",data_sp500)

data_sp500.sort_values("date",ascending=False,inplace=True)
data_sp500=data_sp500.reset_index(drop=True)

0
5
10
15
20
25
30
35
40
45
50
55
60


### CSV final indice du VIX 

In [102]:
data_vix.to_csv("Data/data_vix.csv",index=False)
data_vix.head()

,dernier_VIX,ouverture,higher,lower,variation,date
0,24.98,24.40,25.05,24.18,+9.42%,2022-12-12
1,24.40,22.83,24.40,22.83,6.43%,2022-12-11
2,24.40,22.83,24.40,22.83,6.43%,2022-12-10
3,22.83,22.55,23.21,22.18,+2.42%,2022-12-09
4,22.29,22.81,23.28,22.06,-1.72%,2022-12-08


### CSV final indice de performance du SP500

In [103]:
data_sp500.to_csv("Data/data_sp500.csv",index=False)
data_sp500.head()

,dernier_SP500,ouverture,higher,lower,variation,date
0,3980.20,3943.00,3987.90,3936.50,+1.16%,2022-12-12
1,3943.00,3934.38,3943.00,3934.38,0.22%,2022-12-11
2,3943.00,3934.38,3943.00,3934.38,0.22%,2022-12-10
3,3934.38,3954.17,3977.02,3933.04,-0.73%,2022-12-09
4,3963.51,3947.79,3974.19,3935.83,+0.75%,2022-12-08
